In [57]:
import re
import time
import requests

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import pymorphy3 # работа с русским языком, pymorphy3

import texttable as tt

In [2]:
# рассмотрим данные прогноза погоды в Челябинске (Советский район города)
url = 'https://yandex.ru/pogoda/month/january?lat={latitude}&lon={lontitude}'
url = url.format(latitude=55.159897, lontitude=61.402554)
page = requests.get(url)
bs = BeautifulSoup(page.text, 'html.parser')
print(bs)

<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Погода в январе. Советский район: Яндекс Погода </title><script nonce="1698732851783716-3324213513881617476-balancer-l7leveler-kubr-yp-vla-120-BAL">;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);;(function(d,e,p){e=d.documentElement;p="placeholder";e.className+=" i-ua_"+p+(p in d.createElement("input")?"_yes":"_no")})(document);</script><link href="https://yastat

In [61]:
# информация о погоде за Январь 2023

morph = pymorphy3.MorphAnalyzer(lang='ru')

january = []
temp_january = bs.find('table', {'class': 'climate-calendar'}).find_all('td', {'class': 'climate-calendar__cell'})
for i, val in enumerate(temp_january):
  temp = []
  if val.find('div', {'class': 'climate-calendar-day_colorless_yes'}):
    continue
  temp.append(int(val.find('div', {'class': 'climate-calendar-day__detailed-container-center'}).find_next('h6').text.split(',', 1)[0].split(' ', 1)[0]))
  temp.append(str(morph.parse(val.find('div', {'class': 'climate-calendar-day__detailed-container-center'}).find_next('h6').text.split(',', 1)[0].split(' ', 1)[1])[0].normal_form))
  temp.append(2023)
  temp.append(str(val.find('div', {'class': 'climate-calendar-day__detailed-container-center'}).find_next('h6').text.split(',', 1)[1]))
  temp.append(int(val.find('div', {'class': 'temp climate-calendar-day__detailed-basic-temp-day'}).find('span').text.replace('−', '-')))
  temp.append(int(val.find('div', {'class': 'temp climate-calendar-day__detailed-basic-temp-night'}).find('span').text.replace('−', '-')))
  temp.append(int(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[0].text.split(' ', 1)[0]))
  temp.append(int(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[1].text.split('%', 1)[0]) / 100)
  temp.append(float(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[2].find('div').text))
  temp.append(str(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[2].find('abbr').text))
  january.append(temp)
january

[[1, 'январь', 2023, ' вс', -11, -16, 746, 0.85, 5.5, 'ЮЗ'],
 [2, 'январь', 2023, ' пн', -15, -16, 748, 0.85, 4.5, 'Ю'],
 [3, 'январь', 2023, ' вт', -16, -16, 747, 0.82, 5.5, 'Ю'],
 [4, 'январь', 2023, ' ср', -14, -15, 747, 0.82, 4.8, 'ЮЗ'],
 [5, 'январь', 2023, ' чт', -14, -14, 749, 0.83, 4.6, 'ЮЗ'],
 [6, 'январь', 2023, ' пт', -12, -13, 750, 0.86, 5.3, 'ЮЗ'],
 [7, 'январь', 2023, ' сб', -12, -14, 751, 0.86, 4.1, 'ЮЗ'],
 [8, 'январь', 2023, ' вс', -14, -17, 753, 0.82, 4.5, 'ЮЗ'],
 [9, 'январь', 2023, ' пн', -14, -15, 751, 0.8, 4.6, 'Ю'],
 [10, 'январь', 2023, ' вт', -12, -14, 748, 0.83, 4.6, 'ЮЗ'],
 [11, 'январь', 2023, ' ср', -12, -14, 746, 0.86, 4.6, 'ЮЗ'],
 [12, 'январь', 2023, ' чт', -12, -12, 746, 0.85, 5.0, 'ЮЗ'],
 [13, 'январь', 2023, ' пт', -11, -12, 744, 0.86, 4.4, 'ЮЗ'],
 [14, 'январь', 2023, ' сб', -10, -12, 743, 0.82, 4.8, 'ЮЗ'],
 [15, 'январь', 2023, ' вс', -10, -11, 745, 0.83, 4.6, 'ЮЗ'],
 [16, 'январь', 2023, ' пн', -11, -13, 750, 0.82, 4.5, 'ЮЗ'],
 [17, 'январь', 2023,

In [62]:
# Датафрейм из массива
df = pd.DataFrame(january, columns=['День', 'Месяц', 'Год', 'День недели', 'Температура днем', 'Температура ночью', 'Давление (мм рт. ст.)', 'Влажность (%)', 'Скорость ветра', 'Направление ветра'])
df

День   Месяц   Год День недели  Температура днем  Температура ночью  \
0      1  январь  2023          вс               -11                -16   
1      2  январь  2023          пн               -15                -16   
2      3  январь  2023          вт               -16                -16   
3      4  январь  2023          ср               -14                -15   
4      5  январь  2023          чт               -14                -14   
5      6  январь  2023          пт               -12                -13   
6      7  январь  2023          сб               -12                -14   
7      8  январь  2023          вс               -14                -17   
8      9  январь  2023          пн               -14                -15   
9     10  январь  2023          вт               -12                -14   
10    11  январь  2023          ср               -12                -14   
11    12  январь  2023          чт               -12                -12   
12    13  январь  2023          пт               -11                -12   
13    14  январь  2023          сб               -10                -12   
14    15  январь  2023          вс               -10                -11   
15    16  январь  2023          пн               -11                -13   
16    17  январь  2023          вт               -12                -14   
17    18  январь  2023          ср               -12                -15   
18    19  январь  2023          чт               -14                -15   
19    20  январь  2023          пт               -15                -16   
20    21  январь  2023          сб               -12                -13   
21    22  январь  2023          вс               -12                -14   
22    23  январь  2023          пн               -15                -16   
23    24  январь  2023          вт               -14                -15   
24    25  январь  2023          ср               -13                -14   
25    26  январь  2023          чт               -13                -14   
26    27  январь  2023          пт               -12                -15   
27    28  январь  2023          сб               -14                -15   
28    29  январь  2023          вс               -14                -16   
29    30  январь  2023          пн               -14                -16   
30    31  январь  2023          вт               -13                -14   

    Давление (мм рт. ст.)  Влажность (%)  Скорость ветра Направление ветра  
0                     746           0.85             5.5                ЮЗ  
1                     748           0.85             4.5                 Ю  
2                     747           0.82             5.5                 Ю  
3                     747           0.82             4.8                ЮЗ  
4                     749           0.83             4.6                ЮЗ  
5                     750           0.86             5.3                ЮЗ  
6                     751           0.86             4.1                ЮЗ  
7                     753           0.82             4.5                ЮЗ  
8                     751           0.80             4.6                 Ю  
9                     748           0.83             4.6                ЮЗ  
10                    746           0.86             4.6                ЮЗ  
11                    746           0.85             5.0                ЮЗ  
12                    744           0.86             4.4                ЮЗ  
13                    743           0.82             4.8                ЮЗ  
14                    745           0.83             4.6                ЮЗ  
15                    750           0.82             4.5                ЮЗ  
16                    754           0.82             5.4                ЮЗ  
17                    754           0.83             5.0                ЮЗ  
18                    753           0.81             4.5                 Ю  
19                    751           0.83             5.4       

In [63]:
# cоздаем объект Texttable
tab = tt.Texttable()

# устанавливаем стили таблицы
tab.set_cols_align(['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'])
tab.header(['День', 'Месяц', 'Год', 'День недели', 'Температура днем', 'Температура ночью', 'Давление (мм рт. ст.)', 'Влажность (%)', 'Скорость ветра', 'Направление ветра'])

# преобразуем DataFrame в список списков (двумерный список)
data_list = df.values.tolist()

# добавляем данные в таблицу
for row in data_list:
  tab.add_row(row)

# получаем отформатированную таблицу в виде строки
table_string = tab.draw()

# выводим таблицу на экран
print(table_string)

+------+--------+------+-------+-------+-------+-------+-------+-------+-------+
| День | Месяц  | Год  | День  | Темпе | Темпе | Давле | Влажн | Скоро | Напра |
|      |        |      | недел | ратур | ратур |  ние  | ость  |  сть  | влени |
|      |        |      |   и   |   а   |   а   |  (мм  |  (%)  | ветра |   е   |
|      |        |      |       | днем  | ночью |  рт.  |       |       | ветра |
|      |        |      |       |       |       | ст.)  |       |       |       |
+======+========+======+=======+=======+=======+=======+=======+=======+=======+
|  1   | январь | 2023 |   вс  |  -11  |  -16  |  746  | 0.850 | 5.500 |  ЮЗ   |
+------+--------+------+-------+-------+-------+-------+-------+-------+-------+
|  2   | январь | 2023 |   пн  |  -15  |  -16  |  748  | 0.850 | 4.500 |   Ю   |
+------+--------+------+-------+-------+-------+-------+-------+-------+-------+
|  3   | январь | 2023 |   вт  |  -16  |  -16  |  747  | 0.820 | 5.500 |   Ю   |
+------+--------+------+----